In [81]:
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [82]:
df_train = pd.read_csv('../data/iith_foml_2023_train.csv')
drop_cols = ['Feature 3 (Discrete)', 'Feature 11', 'Feature 10', 'Feature 23 (Discrete)']
df_train.drop(columns=drop_cols, inplace=True)
for column in df_train.columns:
    mean_value = df_train[column].mean()
    df_train[column].fillna(mean_value, inplace=True)

In [83]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [84]:
from sklearn.ensemble import VotingClassifier, StackingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, make_scorer

def report_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro', zero_division=0.0)

# Initialize models
extratrees_model = ExtraTreesClassifier(random_state=42, bootstrap=False, n_jobs=-1)
randomforest_model = RandomForestClassifier(random_state=42, bootstrap=False, n_jobs=-1)
gradientboost_model = GradientBoostingClassifier(random_state=42)
bagging_model = BaggingClassifier(random_state=42, n_jobs=-1)
knn_model = KNeighborsClassifier(n_jobs=-1)
knn_model_2 = KNeighborsClassifier(n_jobs=-1)

model = StackingClassifier(
    estimators=[
        ('gradientboost_model', gradientboost_model),
        # ('randomforest_model', randomforest_model),
        ('extratrees_model', extratrees_model),
        # ('knn_model', knn_model),
        # ('knn_model_2', knn_model_2),
        # ('bagging_model', bagging_model)
    ], 
    n_jobs=-1,
    final_estimator=knn_model,
)

# Cross validate the model
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train, y_train, test_size=0.2)
model.fit(X_train_cv, y_train_cv)
y_pred_cv = model.predict(X_test_cv)
macro_f1 = f1_score(y_test_cv, y_pred_cv, average='macro')
print(f'Macro F1 Score: {macro_f1}')

Macro F1 Score: 0.4173245806621586
